In [1]:
#!/usr/bin/python3.3
# -*-coding:Utf-8 -*

import pandas as pd 
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt 
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'
import xgboost as xgb
import numpy as np

# reading data
train = pd.read_csv("sources/train.csv")
test = pd.read_csv("sources/test.csv")

In [2]:
#add a new column 'date' convert into datetime for both training and test set
train['date']=0
for i in range(len(train)) :
	train['date'][i]=datetime.strptime(train['datetime'][i],'%Y-%m-%d %H:%M:%S')

train=train.set_index("datetime")


test['date']=0
for i in range(len(test)) :
	test['date'][i]=datetime.strptime(test['datetime'][i],'%Y-%m-%d %H:%M:%S')

test=test.set_index("datetime")

In [3]:
#create day of week column and hour column and a year colum for both training and test set
train['day']=0
train['hour']=0
train['year']=0
for i in range(len(train)) :
    train['day'][i] = train['date'][i].weekday()
    train['hour'][i] = train['date'][i].hour
    train['year'][i]=train['date'][i].year

test['day']=0
test['hour']=0
test['year']=0
for i in range(len(test)) :
    test['day'][i] = test['date'][i].weekday()
    test['hour'][i] = test['date'][i].hour
    test['year'][i]=test['date'][i].year


In [4]:
#create weekEnd variable 
train['weekEnd'] = 0 ;
train.ix[train['day']>=5,'weekEnd']=1

test['weekEnd'] = 0 ;
test.ix[test['day']>=5,'weekEnd']=1


In [5]:
'''
#create ahot () variable for both training and test set
train['ahot'] = 4
for i in range(len(train)) :
	currentTemp = train['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		train['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		train['ahot'][i] = 2
	elif ( currentTemp<16.6) :
		train['ahot'][i] = 1


test['ahot'] = 4
for i in range(len(test)) :
	currentTemp = test['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		test['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		test['ahot'][i] = 2
	elif (currentTemp<16.6) :
		test['ahot'][i] = 1
'''


"\n#create ahot () variable for both training and test set\ntrain['ahot'] = 4\nfor i in range(len(train)) :\n\tcurrentTemp = train['atemp'][i]\n\tif (currentTemp>=24.24 and currentTemp<31.06 ):\n\t\ttrain['ahot'][i] = 3\n\telif (currentTemp>=16.6 and currentTemp<24.24):\n\t\ttrain['ahot'][i] = 2\n\telif ( currentTemp<16.6) :\n\t\ttrain['ahot'][i] = 1\n\n\ntest['ahot'] = 4\nfor i in range(len(test)) :\n\tcurrentTemp = test['atemp'][i]\n\tif (currentTemp>=24.24 and currentTemp<31.06 ):\n\t\ttest['ahot'][i] = 3\n\telif (currentTemp>=16.6 and currentTemp<24.24):\n\t\ttest['ahot'][i] = 2\n\telif (currentTemp<16.6) :\n\t\ttest['ahot'][i] = 1\n"

In [6]:
'''
#create ahot () variable for both training and test set
train['hot'] = 4
for i in range(len(train)) :
	currentTemp = train['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		train['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		train['hot'][i] = 2
	elif ( currentTemp<13.94) :
		train['hot'][i] = 1


test['hot'] = 4
for i in range(len(test)) :
	currentTemp = test['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		test['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		test['hot'][i] = 2
	elif (currentTemp<13.94) :
		test['hot'][i] = 1
'''


"\n#create ahot () variable for both training and test set\ntrain['hot'] = 4\nfor i in range(len(train)) :\n\tcurrentTemp = train['temp'][i]\n\tif (currentTemp>=20.5 and currentTemp<26.24 ):\n\t\ttrain['hot'][i] = 3\n\telif (currentTemp>=13.94 and currentTemp<20.5):\n\t\ttrain['hot'][i] = 2\n\telif ( currentTemp<13.94) :\n\t\ttrain['hot'][i] = 1\n\n\ntest['hot'] = 4\nfor i in range(len(test)) :\n\tcurrentTemp = test['temp'][i]\n\tif (currentTemp>=20.5 and currentTemp<26.24 ):\n\t\ttest['hot'][i] = 3\n\telif (currentTemp>=13.94 and currentTemp<20.5):\n\t\ttest['hot'][i] = 2\n\telif (currentTemp<13.94) :\n\t\ttest['hot'][i] = 1\n"

In [7]:
train = train.reset_index()
train = train.drop("datetime",1)
train = train.drop(['date'],1)

test = test.reset_index()
result= test[['datetime']]

test = test.drop("datetime",1)
test = test.drop('date',1)

In [8]:
train_registered=train.drop("casual",1)
train_registered=train_registered.drop("count",1)

train_casual=train.drop("registered",1)
train_casual=train_casual.drop("count",1)

# Cross validation

In [9]:
X_train = train_registered.drop("registered",1)
Y_train = train_registered['registered']
DTrain_xgb = xgb.DMatrix(X_train, Y_train)

In [10]:
x_parameters = { 'max_depth':10, 'subsample':0.8,"min_child_weight":6,"scale_pos_weight":5}
cross = xgb.cv(x_parameters, DTrain_xgb,
              num_boost_round=25
              )
print(cross.describe())
cross

/user/3/fischmaa/anaconda3/lib/python3.5/site-packages/xgboost/training.py:272: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  idset = [randidx[(i * kstep): min(len(randidx), (i + 1) * kstep)] for i in range(nfold)]


       test-rmse-mean  test-rmse-std  train-rmse-mean  train-rmse-std
count       25.000000      25.000000        25.000000       25.000000
mean        48.150214       1.566835        36.667143        0.496941
std         29.577638       0.472801        34.377700        0.159122
min         34.552611       0.572962        15.938119        0.276322
25%         34.575433       1.377960        18.137121        0.344774
50%         35.026765       1.772239        21.940766        0.440727
75%         42.309537       1.911411        34.975374        0.615746
max        156.102651       1.938511       155.886470        0.769889


,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,156.102651,0.704347,155.886470,0.362065
1,114.788877,0.654564,113.750893,0.331120
2,86.183988,0.572962,84.434728,0.440727
3,67.168503,0.913733,64.388972,0.517825
4,55.001158,0.847197,50.610117,0.610968
5,47.250683,1.028237,41.310777,0.615504
6,42.309537,1.377960,34.975374,0.600519
7,39.256007,1.533675,30.740060,0.616302
8,37.492783,1.666558,27.908003,0.615746
9,36.439222,1.721226,25.875664,0.708984


In [11]:
cross.describe()

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
count,25.000000,25.000000,25.000000,25.000000
mean,48.150214,1.566835,36.667143,0.496941
std,29.577638,0.472801,34.377700,0.159122
min,34.552611,0.572962,15.938119,0.276322
25%,34.575433,1.377960,18.137121,0.344774
50%,35.026765,1.772239,21.940766,0.440727
75%,42.309537,1.911411,34.975374,0.615746
max,156.102651,1.938511,155.886470,0.769889


In [12]:
X_train = train_casual.drop("casual",1)
Y_train = train_casual['casual']
Dtrain_xgb = xgb.DMatrix(X_train, Y_train)

In [13]:
x_parameters = { 'max_depth':8, 'subsample':0.8,"min_child_weight":5,"scale_pos_weight":5}
cross = xgb.cv(x_parameters, DTrain_xgb,
              num_boost_round=20
              )
print(cross.describe())
cross

/user/3/fischmaa/anaconda3/lib/python3.5/site-packages/xgboost/training.py:272: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  idset = [randidx[(i * kstep): min(len(randidx), (i + 1) * kstep)] for i in range(nfold)]


       test-rmse-mean  test-rmse-std  train-rmse-mean  train-rmse-std
count       20.000000      20.000000        20.000000       20.000000
mean        52.708764       0.926748        45.779011        0.604598
std         32.610506       0.503183        35.778767        0.235094
min         34.509060       0.180761        22.011019        0.306300
25%         34.915112       0.423008        24.542096        0.418397
50%         37.502183       1.018639        30.116602        0.561439
75%         51.675134       1.403030        48.061700        0.756735
max        157.059952       1.579511       156.640777        1.106588


,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,157.059952,0.580997,156.640777,0.420716
1,116.551145,0.180761,115.454386,0.306300
2,88.290754,0.187734,86.933302,0.337748
3,69.653035,0.354065,67.555934,0.477131
4,57.453007,0.423912,54.618459,0.779138
5,49.749176,0.370678,45.876114,1.106588
6,44.442545,0.420297,39.658444,0.961986
7,41.512206,0.470089,35.886087,1.012924
8,39.270176,0.930930,32.907520,0.585736
9,38.011735,1.069206,30.992136,0.537142


In [14]:
cross.describe()

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
count,20.000000,20.000000,20.000000,20.000000
mean,52.708764,0.926748,45.779011,0.604598
std,32.610506,0.503183,35.778767,0.235094
min,34.509060,0.180761,22.011019,0.306300
25%,34.915112,0.423008,24.542096,0.418397
50%,37.502183,1.018639,30.116602,0.561439
75%,51.675134,1.403030,48.061700,0.756735
max,157.059952,1.579511,156.640777,1.106588


# training

In [15]:
X_train = train_registered.drop("registered",1)
Y_train = train_registered['registered']
DTrain_xgb = xgb.DMatrix(X_train, Y_train)
params = {}
params["objective"] = "reg:linear"
params["min_child_weight"] = 6
params["scale_pos_weight"] = 5
params["max_depth"] = 9
params["booster"] = "gbtree"
params["nthread"] = 8
params["tree_method"] = "exact"
gbm_reg = xgb.train(dtrain=DTrain_xgb, params = params,num_boost_round=7)
X_test = xgb.DMatrix(test)
Y_pred_reg = gbm_reg.predict(X_test)
print(Y_pred_reg)
xgb.plot_importance(gbm_reg)


[  9.22056389   4.23569918   1.98947811 ...,  98.02184296  93.66797638
  38.43677902]


In [16]:
plt.show()

In [17]:
X_train = train_casual.drop("casual",1)
Y_train = train_casual['casual']
Dtrain_xgb = xgb.DMatrix(X_train, Y_train)
params = {}
params["objective"] = "reg:linear"
params["min_child_weight"] = 5
params["scale_pos_weight"] = 5
params["max_depth"] = 8
params["booster"] = "gbtree"
params["nthread"] = 8
params["tree_method"] = "exact"
gbm_cas = xgb.train(dtrain=Dtrain_xgb, params = params, num_boost_round=7)
X_test = xgb.DMatrix(test)
Y_pred_cas = gbm_cas.predict(X_test)
print(Y_pred_cas)
xgb.plot_importance(gbm_cas)


[ 1.45737624  0.79732108  0.85412222 ...,  3.47965431  2.20436287
  1.24872303]


In [18]:
plt.show()

# Try to boost model 

In [19]:
'''
# Using 6000 rows for training and  4000 validation separation
offset = 6000
X_train = train_registered.drop("registered",1)
Y_train = train_registered[['registered']]

xgtrain = xgb.DMatrix(X_train.ix[:offset, :], label=Y_train.ix[:offset])
xgval = xgb.DMatrix(X_train.ix[offset:, :], label=Y_train.ix[offset:])
num_round=10000
params={}
params["objective"] = "reg:linear"
#params["min_child_weight"] = 6
#params["scale_pos_weight"] = 5
#params["max_depth"] = [7,8,9,10,11]
#params["booster"] = "gbtree"
#params["nthread"] = 8
#params["tree_method"] = "exact"
Dtrain_xgb = xgb.DMatrix(X_train, Y_train)
params = {}

evallist = [(xgtrain, 'train'), (xgval, 'val')]

gmb_reg = xgb.train(dtrain=xgtrain, evals=evallist, params=params, num_boost_round=num_round, early_stopping_rounds=100)

X_test = xgb.DMatrix(test)
Y_pred_reg = gbm_reg.predict(X_test, ntree_limit=gmb_reg.best_iteration)


# Using 6000 rows for training and  4000 validation separation
offset = 6000
X_train = train_casual.drop("casual",1)
Y_train = train_casual[['casual']]
xgtrain = xgb.DMatrix(X_train.ix[:offset, :], label=Y_train.ix[:offset])
xgval = xgb.DMatrix(X_train.ix[offset:, :], label=Y_train.ix[offset:])
num_round=10000

Dtrain_xgb = xgb.DMatrix(X_train, Y_train)
params = {}
params["objective"] = "reg:linear"
params["min_child_weight"] = 5
params["scale_pos_weight"] = 5
params["max_depth"] = [8,9,10,11,12]
params["booster"] = "gbtree"
params["nthread"] = 8
params["tree_method"] = "exact"
evallist = [(xgtrain, 'train'), (xgval, 'val')]

gmb_cas = xgb.train(dtrain=xgtrain, evals=evallist, params=params, num_boost_round=num_round, early_stopping_rounds=100)

X_test = xgb.DMatrix(test)
Y_pred_cas = gbm_cas.predict(X_test, ntree_limit=gmb_cas.best_iteration)
#xgb.plot_importance(gbm_cas)
'''

'\n# Using 6000 rows for training and  4000 validation separation\noffset = 6000\nX_train = train_registered.drop("registered",1)\nY_train = train_registered[[\'registered\']]\n\nxgtrain = xgb.DMatrix(X_train.ix[:offset, :], label=Y_train.ix[:offset])\nxgval = xgb.DMatrix(X_train.ix[offset:, :], label=Y_train.ix[offset:])\nnum_round=10000\nparams={}\nparams["objective"] = "reg:linear"\n#params["min_child_weight"] = 6\n#params["scale_pos_weight"] = 5\n#params["max_depth"] = [7,8,9,10,11]\n#params["booster"] = "gbtree"\n#params["nthread"] = 8\n#params["tree_method"] = "exact"\nDtrain_xgb = xgb.DMatrix(X_train, Y_train)\nparams = {}\n\nevallist = [(xgtrain, \'train\'), (xgval, \'val\')]\n\ngmb_reg = xgb.train(dtrain=xgtrain, evals=evallist, params=params, num_boost_round=num_round, early_stopping_rounds=100)\n\nX_test = xgb.DMatrix(test)\nY_pred_reg = gbm_reg.predict(X_test, ntree_limit=gmb_reg.best_iteration)\n\n\n# Using 6000 rows for training and  4000 validation separation\noffset = 6

In [20]:
result['count'] = pd.DataFrame( Y_pred_cas + Y_pred_reg)

result_final = result.set_index('datetime')
result_final[result_final['count']<=0]=0
result_final

,count
datetime,
2011-01-20 00:00:00,10.677940
2011-01-20 01:00:00,5.033020
2011-01-20 02:00:00,2.843600
2011-01-20 03:00:00,1.880065
2011-01-20 04:00:00,1.880065
2011-01-20 05:00:00,7.178566
2011-01-20 06:00:00,33.443798
2011-01-20 07:00:00,91.531502
2011-01-20 08:00:00,193.241180


In [21]:
result_final.to_csv('Résultats/XGBoost/submissionXGBoost.csv')